# Car Detection Tutorial

We will learn how to detect cars on the road in a video using Intel Distribution of OpenVINO toolkit and MobileNet-SSD model. First, we will download and optimize the model with OpenVINO Model Optimizer. We will use OpenVINO Inference Engine Async API to load the network into target devices via plugins. We will utilize such hardware targets as Intel® HD Graphics GPU, Intel® Movidius™ Neural Compute Stick 2, and Intel® Arria® 10 FPGA. We will use OpenCV for video handling and creating bounding boxes and labels for the output. At the end, we will watch the output videos with cars detected and construct a performance comparison graph.  

## Steps

We will start with running inference on a single image to see how the Intel® Distribution of OpenVINO™ toolkit works, then we will run inference on a video stream.

We will go over detecting cars with OpenVINO in several steps:

1. Create an intermediate representation (IR) of the model using the Model Optimizer.
2. Create IEPlugin for the device.
3. Read the model's IR using IENetwork.
4. Load the IENetwork instance into the plugin.
5. Preprocess image and run inference on it.
6. Create a job file to target different hardware types.
7. Submit jobs to the queue.
8. View the results and hardware performance comparison.

## Create an Intermediate Representation of the Model

Model Optimizer creates the Intermediate Representation of the model which is the device-agnostic, generic optimization of the model. Caffe\*, TensorFlow\*, MXNet\*, ONNX\*, and Kaldi\* models are supported by Model Optimizer.

We will use the **MobileNet-SSD** model. Download the model, specifying the name and output directory.

In [ ]:
!/opt/intel/openvino/deployment_tools/tools/model_downloader/downloader.py --name mobilenet-ssd -o raw_models

Let's convert this model to the intermediate representation using Model Optimizer.

In [ ]:
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel \
--data_type FP32 \
--output_dir models/mobilenet-ssd/FP32 \
--scale 256 \
--mean_values [127,127,127] 

We will also need the FP16 IR version of the model for the calculations on Neural Compute Stick 2 (NCS2) and Visual Processing Unit (VPU); let's create it here.

In [ ]:
!/opt/intel/openvino/deployment_tools/model_optimizer/mo.py \
--input_model raw_models/object_detection/common/mobilenet-ssd/caffe/mobilenet-ssd.caffemodel \
--data_type FP16 \
--output_dir models/mobilenet-ssd/FP16 \
--scale 256 \
--mean_values [127,127,127] 

## Input Video

Run the following cell to create a symlink and view the input video.

In [ ]:
!ln -sf /data/reference-sample-data/object-detection-python/cars_1900.mp4 
videoHTML('Cars video', ['cars_1900.mp4'])

## Import dependencies

In [ ]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import os
import time
import sys                                     
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *
from openvino.inference_engine import IEPlugin, IENetwork
import cv2
# For labeling the image
from out_process import placeBoxes

## Create Plugin

Let's create a function to construct a plugin.

In [ ]:
def createPlugin(device, extension_list):
    # Plugin initialization for specified device. We will be targeting CPU initially.
    plugin = IEPlugin(device=device)

    # Loading additional extension libraries for the CPU
    for extension in extension_list:
        plugin.add_cpu_extension('/opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so')
    
    return plugin

## Read the IR of the Model

Let's import the optimized model into our neural network using **`IENetwork`**. 


In [ ]:
def createNetwork(model_xml, model_bin, plugin):
    # Importing network weights from IR models.
    net = IENetwork(model=model_xml, weights=model_bin)
    
    # Some layers in IR models may be unsupported by some plugins. 
    if "CPU" in plugin.device:
        supported_layers = plugin.get_supported_layers(net)
        not_supported_layers = [l for l in net.layers.keys() if l not in supported_layers]
        if len(not_supported_layers) != 0:
            print("Following layers are not supported by the plugin for specified device {}:\n {}".
                      format(plugin.device, ', '.join(not_supported_layers)))
            print("Please try to specify cpu extensions library path in sample's command line parameters "
                  "using -l or --cpu_extension command line argument")
            return None
    return net

## Load the Network into the Plugin

Once we have the plugin and the network, we can load the network into the plugin.

In [ ]:
def loadNetwork(plugin, net):
    # Loading IR model to the plugin.
    exec_net = plugin.load(network=net, num_requests=2)
    
    # Getting the input and outputs of the network
    input_blob = next(iter(net.inputs))
    out_blob = next(iter(net.outputs))
    return exec_net,input_blob,out_blob

## Preprocess Image

We will need the function to load the image using OpenCV and change its shape to be compatible with our network.

In [ ]:
def preprocessImage(img_path, net, input_blob):
    # Reading the frame from a jpeg file
    frame = cv2.imread(img_path)
    
    # Reshaping data
    n, c, h, w = net.inputs[input_blob].shape
    in_frame = cv2.resize(frame, (w, h))
    in_frame = in_frame.transpose((2, 0, 1))  # Change data layout from HWC to CHW
    return in_frame.reshape((n, c, h, w)),frame

## Run Inference on a Single Frame

Now we are ready to run the inference workload using the plugin. We will be running in **async_mode** by using `start_async` method. With the async_mode, the inference is started in parallel on either a separate thread or device.
In other words, `start_async` is non-blocking and the main process is free to do any additional processing needed. 
In the next section, we will see an implementation of pipelining to mask the latency of loading and modifying images.

During asynchronous runs, the different images are tracked by an integer `request_id`. Because we only have one image to process, we will just use 0.

In [ ]:
def runInference():
    plugin = createPlugin(device='CPU', extension_list=['/data/reference-sample-data/extension/libcpu_extension.so'])
    model_xml = "models/mobilenet-ssd/FP32/mobilenet-ssd.xml"
    model_bin = "models/mobilenet-ssd/FP32/mobilenet-ssd.bin"
    net = createNetwork(model_xml, model_bin, plugin)
    exec_net,input_blob,out_blob = loadNetwork(plugin, net)
    in_frame,original_frame = preprocessImage('cars_1900_first_frame.jpg', net, input_blob)
    
    my_request_id=0

    # Starting the inference in async mode, which starts the inference in parallel
    exec_net.start_async(request_id=my_request_id, inputs={input_blob: in_frame})
    # ... You can do additional processing or latency masking while we wait ...

    # Blocking wait for a particular request_id
    if exec_net.requests[my_request_id].wait(-1) == 0:
        # getting the result of the network
        res = exec_net.requests[my_request_id].outputs[out_blob]
        
        # Processing the output result and adding labels on the image. Implementation is not shown in the
        #  this notebook; you can find it in object_detection_demo_ssd_async.py
        prob_threshold = 0.5  # 50% confidence needed for "detection"
        initial_w = original_frame.shape[1]
        initial_h = original_frame.shape[0]
        frame = placeBoxes(res, None, prob_threshold, original_frame, initial_w, initial_h, False, my_request_id, 0)
        fig = plt.figure(dpi=300)
        ax = fig.add_subplot(111)
        ax.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), interpolation='none')
        plt.axis("off")
        plt.show()
    else:
        print("There was an error with the request")

runInference()

## How Video Inferencing Works

Video inferencing is similar to the image inferencing, however, there are few differences. Let's review them.

The following lines determine the source of the video. We will use a pre-recorded input video file in this example, we could also use a camera by setting the input argument to 'cam'.

```python
if args.input == 'cam':
        input_stream = 0
        out_file_name = 'cam'
    else:
        input_stream = args.input
```

We capture frames from the video sample using **OpenCV VideoCapture** API.

```python
cap = cv2.VideoCapture(input_stream)
```

Also, we have a latency-masking scheme, where we post-process a frame while another frame is being processed on the inference engine.

```python
cur_request_id = 0
next_request_id = 1

while cap.isOpened():
   # ... load next frame from cap ...

   # start the next frame
   exec_net.start_async(request_id=next_request_id, inputs={input_blob: in_frame})

   # see if the current frame is ready
   if exec_net.requests[cur_request_id].wait(-1) == 0:
       # ... post-processing current frame ...
    
   # swap request ids
   cur_request_id, next_request_id = next_request_id, cur_request_id
```

These code snippets are parts of <a href="object_detection_demo_ssd_async.py">object_detection_demo_ssd_async.py</a> which we will call via running a job.

## Create a Job File

All the code up to this point has been run within the Jupyter Notebook instance running on a development node based on an Intel® Xeon® Scalable Processor, where the Notebook is allocated a single core. To run inference on the entire video, we need more compute power. We will run the workload on several DevCloud's edge compute nodes. We will send work to the edge compute nodes by submitting jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

To pass the specific variables to the Python code, we will use following arguments:

* `-m`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;location of the optimized **MobileNet-SSD** model's XML
* `-i`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;location of the input video
* `-o`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output directory
* `-d`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;hardware device type (CPU, GPU, MYRIAD, HDDL or HETERO:FPGA,CPU)
* `-l`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;path to the CPU extension library

The job file will be executed directly on the edge compute node.

In [ ]:
%%writefile object_detection_job.sh

# The default path for the job is your home directory, so we change directory to where the files are.
cd $PBS_O_WORKDIR

# Object detection script writes output to a file inside a directory. We make sure that this directory exists.
# The output directory is the first argument of the bash script
OUTPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3
INPUT_FILE=$4

if [ "$DEVICE" = "HETERO:FPGA,CPU" ]; then
    # Environment variables and compilation for edge compute nodes with FPGAs
    source /opt/fpga_support_files/setup_env.sh
    aocl program acl0 /opt/intel/openvino/bitstreams/a10_vision_design_bitstreams/2019R1_PL1_FP11_MobileNet_Clamp.aocx
fi
    
# Running the object detection code
SAMPLEPATH=$PBS_O_WORKDIR
python3 object_detection_demo_ssd_async.py  -m ${SAMPLEPATH}/models/mobilenet-ssd/${FP_MODEL}/mobilenet-ssd.xml \
                                            -i $INPUT_FILE \
                                            -o $OUTPUT_FILE \
                                            -d $DEVICE \
                                            -l /opt/intel/openvino/deployment_tools/inference_engine/lib/intel64/libcpu_extension_avx2.so

g++ -std=c++14 ROI_writer.cpp -o ROI_writer  -lopencv_core -lopencv_videoio -lopencv_imgproc -lopencv_highgui  -fopenmp -I/opt/intel/openvino/opencv/include/ -L/opt/intel/openvino/opencv/lib/

# Rendering the output video
SKIPFRAME=1
RESOLUTION=0.5
./ROI_writer $INPUT_FILE $OUTPUT_FILE $SKIPFRAME $RESOLUTION

### Optional Change of Video Quality

Video rendering is a separate task invoked by `ROI_writer` at the end of the job. To reduce rendering time, you can reduce the output video quality using the SKIP_FRAME and RESOLUTION variables. 

`SKIP_FRAME=1` writes all processed video frames with bounding boxes into the output video. This is the slowest option and it preserves all inference data in the output video stream. `SKIP_FRAME=2` writes every other frame of the processed frames into the output video.

`RESOLUTION=1` produces the output video with the same resolution as the input video; this is the slowest option.
`RESOLUTION<1` reduces the output video resolution. Here we have `RESOLUTION=0.5` which sets the output video resolution in each dimension to 50% of the input video's resolution.

## How to Submit Jobs to the Queue

Now we can submit the jobs to edge compute nodes. In the DevCloud, you can do this using the `qsub` command.
We can submit object_detection_job to several different types of edge compute nodes simultaneously or just one node at a time.

There are three options of `qsub` command:
- `-l`: the number and the type of nodes in a form of `nodes={node_count}:{property}` 
- `-F`: arguments to pass to `object_detection_job.sh` 
- `-N`: unique job name

The [object_detection_job.sh](object_detection_job.sh) takes in 4 arguments:
1. the path to the directory for the output video and performance stats
2. target device
3. the floating precision point
4. the path to the input video

Run the cell below to view the available types of nodes on the DevCloud. Here you can see the number of available nodes and properties.

In [ ]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

## Set the Video for Inferencing

In [ ]:
os.environ["VIDEO"] = "/data/reference-sample-data/object-detection-python/cars_1900.mp4"

## Submit Jobs to the Queue

Each of the cells below will submit a job targeting different edge compute nodes.
The output of the `qsub` function is the `job_id` which you can use to track progress of a specific job. `qsub` submits job into the queue and a job runs as soon as the requested compute resource becomes available. 


### Intel® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel® Core™ i5-6500TE processor</a>. The inference workload will run the CPU.

In [ ]:
#Submit job to the queue
job_id_core = !qsub object_detection_job.sh -l nodes=1:tank-870:i5-6500te -F "results/ CPU FP32 $VIDEO" -N obj_det_core
print(job_id_core[0]) 
#Progress indicators
if job_id_core:
    progressIndicator('results/', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/', 'v_progress_'+job_id_core[0]+'.txt', "Rendering", 0, 100)
else:                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         
    print("Error in job submission.")

### Intel® Xeon® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz-">Intel® 
    Xeon® Processor E3-1268L v5</a>. The inference workload will run on the CPU.

In [ ]:
#Submit job to the queue
job_id_xeon = !qsub object_detection_job.sh -l nodes=1:tank-870:e3-1268l-v5 -F "results/ CPU FP32 $VIDEO" -N obj_det_xeon 
print(job_id_xeon[0]) 
#Progress indicators
if job_id_xeon:
    progressIndicator('results/', 'i_progress_'+job_id_xeon[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/', 'v_progress_'+job_id_xeon[0]+'.txt', "Rendering", 0, 100)

### Intel® Core CPU with Intel® GPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel® Core i5-6500TE</a>. The inference workload will run on the Intel® HD Graphics 530 card integrated with the CPU.

In [ ]:
#Submit job to the queue
job_id_gpu = !qsub object_detection_job.sh -l nodes=1:tank-870:i5-6500te:intel-hd-530 -F "results/ GPU FP32 $VIDEO" -N obj_det_gpu 
print(job_id_gpu[0]) 
#Progress indicators
if job_id_gpu:
    progressIndicator('results/', 'i_progress_'+job_id_gpu[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/', 'v_progress_'+job_id_gpu[0]+'.txt', "Rendering", 0, 100)

### Intel® Arria® 10 FPGA
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core™ i5-6500te CPU</a> . The inference workload will run on the <a href="https://www.ieiworld.com/mustang-f100/en/"> IEI Mustang-F100-A10 </a> card installed in this node.

In [ ]:
#Submit job to the queue
job_id_fpga = !qsub object_detection_job.sh -l nodes=1:tank-870:i5-6500te:iei-mustang-f100-a10 -F "results/ HETERO:FPGA,CPU FP32 $VIDEO" -N obj_det_fpga
print(job_id_fpga[0]) 
#Progress indicators
if job_id_fpga:
    progressIndicator('results/', 'i_progress_'+job_id_fpga[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/', 'v_progress_'+job_id_fpga[0]+'.txt', "Rendering", 0, 100)

### Intel® Neural Compute Stick 2
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel Core i5-6500te CPU</a>. The inference workload will run on an <a 
    href="https://software.intel.com/en-us/neural-compute-stick">Intel Neural Compute Stick 2</a> installed in this  node.

In [ ]:
#Submit job to the queue
job_id_ncs2 = !qsub object_detection_job.sh -l nodes=1:tank-870:i5-6500te:intel-ncs2 -F "results/ MYRIAD FP16 $VIDEO" -N obj_det_ncs2
print(job_id_ncs2[0]) 
#Progress indicators
if job_id_ncs2:
    progressIndicator('results/', 'i_progress_'+job_id_ncs2[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/', 'v_progress_'+job_id_ncs2[0]+'.txt', "Rendering", 0, 100)

### Intel® Movidius™ Myriad™ X  Vision Processing Unit
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank 870-Q170</a> edge node with an <a href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel® Core i5-6500te CPU</a>. The inference workload will run on an <a 
    href="https://www.ieiworld.com/mustang-v100/en/">IEI Mustang-V100-MX8 </a>accelerator installed in this node.

In [ ]:
#Submit job to the queue
job_id_vpu = !qsub object_detection_job.sh -l nodes=1:tank-870:i5-6500te:iei-mustang-v100-mx8 -F "results/ HDDL FP16 $VIDEO" -N obj_det_vpu
print(job_id_vpu[0]) 
#Progress indicators
if job_id_vpu:
    progressIndicator('results/', 'i_progress_'+job_id_vpu[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/', 'v_progress_'+job_id_vpu[0]+'.txt', "Rendering", 0, 100)

### UP Squared Grove IoT Development Kit
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/up-squared-grove-dev-kit">UP Squared Grove IoT Development Kit</a> edge node with an <a 
    href="https://ark.intel.com/products/96488/Intel-Atom-x7-E3950-Processor-2M-Cache-up-to-2-00-GHz-">Intel® Atom® x7-E3950 Processor</a>. The inference  workload will run on the integrated Intel® HD Graphics 505 card.

In [ ]:
#Submit job to the queue
job_id_up2 = !qsub object_detection_job.sh -l nodes=1:up-squared -F "results/ GPU FP32 $VIDEO" -N obj_det_up2
print(job_id_up2[0]) 
#Progress indicators
if job_id_up2:
    progressIndicator('results/', 'i_progress_'+job_id_up2[0]+'.txt', "Inference", 0, 100)
    progressIndicator('results/', 'v_progress_'+job_id_up2[0]+'.txt', "Rendering", 0, 100)

### Check the Progress

Check the progress of the jobs. `Q` status stands for `queued`, `R` for `running`. How long a job is being queued is dependent on number of the users. It should take up to 5 minutes for a job to run. If the job is no longer listed, it's done. 

In [ ]:
liveQstat()

## View Results

Once the jobs are completed, the queue system outputs the `stdout` and `stderr` streams of each job into files with names
`obj_det_{type}.o{JobID}` and `obj_det_{type}.e{JobID}`. Here, obj_det_{type} corresponds to the `-N` option of qsub. For example, `core` for Core CPU target.

You can find the output video files inside the `results` directory. We wrote a short utility script that will display these videos within the notebook. See `demoutils.py` if you are interested in understanding further how the results are displayed in notebook. 

Run the cell below to display output video from the inference run on Intel Core CPU.

In [ ]:
videoHTML('IEI Tank (Intel Core CPU)', 
          ['results/output_'+job_id_core[0]+'.mp4'], 
          'results/stats_'+job_id_core[0]+'.txt')

## Performance Comparison

The running time of each inference task is recorded in `stats_*job_id*_*architectute*.txt` in `results` folder, where the *architecture* corresponds to the architecture of the target edge compute node. Run the cell below to plot the results of all jobs side-by-side. Lower values for processing time mean better performance. Keep in mind that some architectures are optimized for the highest performance, others for low power or other metrics.

In [ ]:
arch_list = [('core', 'Intel Core\ni5-6500TE\nCPU'),
             ('xeon', 'Intel Xeon\nE3-1268L v5\nCPU'),
             ('gpu', ' Intel Core\ni5-6500TE\nGPU'),
             ('fpga', ' IEI Mustang\nF100-A10\nFPGA'),
             ('ncs2', 'Intel\nNCS2'),
             ('vpu', ' IEI Mustang\nV100-MX8\nVPU'),
             ('up2', 'Intel Atom\nx7-E3950\nUP2/GPU')]

stats_list = []
for arch, a_name in arch_list:
    if 'job_id_'+arch in vars():
        stats_list.append(('results/stats_'+vars()['job_id_'+arch][0]+'.txt', a_name))
    else:
        stats_list.append(('placeholder'+arch, a_name))

summaryPlot(stats_list, 'Architecture', 'Time, seconds', 'Inference Engine Processing Time', 'time' )

summaryPlot(stats_list, 'Architecture', 'Frames per second', 'Inference Engine FPS', 'fps' )